# Data reading and preparation over Time 

In [75]:
# ----------------------data -----------------------------------------#
# CFBP analysis 
import os
os.chdir('C:/Users/Rahil/Documents/Data incubator application/application_Oct_2018/CFBP')

path=os.getcwd()

import pandas as pd


filNames= os.listdir()
data= pd.read_csv(path+'/'+'Consumer_Complaints.csv')


# ----------------------data 2 -----------------------------------------#

data2=data.copy()

data2['Clean_ZIPcode_5d']=data2['ZIP code'].str.extract('^(\d{5})$', expand=True)

data2['Clean_ZIPcode_5d'].isnull().sum(axis=0)
#Out[80]: 472589

len(data2['Clean_ZIPcode_5d'].dropna())
#Out[8]: 672259

data2['Clean_ZIPcode_5_3_d']=data2['ZIP code'].str.extract('^(\d{3,5}X{,2})$', expand=False)
#data2['Clean_ZIPcode_5_3_d'].isnull().sum(axis=0)
#Out[91]: 88798
len(data2['Clean_ZIPcode_5_3_d'].dropna())
pd.notnull(data2['Clean_ZIPcode_5_3_d']).sum(axis=0)
#Out[11]: 1056050

# if using the 3 digits zipcode with XX at the end as well as 5 digits, then use only       
data2['Clean_ZIPcode_5_3_d']=data2['Clean_ZIPcode_5_3_d'].str.slice(start=0, stop=3, step=None)
data2['Clean_ZIPcode_5_3_d'].dropna()
data2['ZipZone']=data2['Clean_ZIPcode_5_3_d'].str.slice(start=0, stop=1, step=None).astype('category')

# ----------------------data _time analysis -----------------------------------------#

#deltaT= pd.to_datetime(data['Date sent to company'], format= '%m/%d/%Y')- pd.to_datetime(data['Date received'], format= '%m/%d/%Y')

time_resp_Analyse=data.copy()[['Timely response?','Submitted via', 'Consumer disputed?', 'Product' ]]
#time_resp_Analyse['deltaTime']=deltaT.dt.days
time_resp_Analyse['ZipZone']=data2.copy()['ZipZone']
time_resp_Analyse['Y_compl']=pd.to_datetime(data['Date sent to company'], format= '%m/%d/%Y').dt.year
time_resp_Analyse['M_compl']=pd.to_datetime(data['Date sent to company'], format= '%m/%d/%Y').dt.month
#time_resp_Analyse['deltaTime'].plot(kind='bar')

season={'Spring':[3,4,5], 'Summer': [6,7,8], 'Fall': [9,10,11], 'Winter': [12,1,2]}
season_s=[]
for m in time_resp_Analyse['M_compl']:
    if m in season['Spring']:
        v='Spring'
    elif m in season['Summer']:
        v='Summer'
    elif m in season['Fall']:
        v='Fall'
    else:
        v='Winter'
    season_s.append(v)   
time_resp_Analyse['Season']=season_s


In [76]:
import datetime
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly.plotly as py
from ipywidgets import widgets
from IPython.display import display, clear_output, Image
from plotly.widgets import GraphWidget
import matplotlib as mpl


## Define function to plot and interact with widgets and parameters

In [78]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual


import seaborn as sns        
sns.set(style="white")  

############### YEAR
GB1= time_resp_Analyse.groupby(['Y_compl', 'Product'])['Y_compl'].count().unstack('Product').fillna(0)

############### SEASON
GB2= time_resp_Analyse.groupby(['Y_compl','Season' ,'Product'])['Season'].count().unstack('Product').fillna(0)
   
############### MONTH
GB3= time_resp_Analyse.groupby(['Y_compl', 'M_compl', 'Product'])['M_compl'].count().unstack('Product').fillna(0)

#################



def plot_ProductComplains_overTime(timeStamp):
    #timeStamp=timeStamp.value
    if timeStamp=='Year':
        data=GB1
    elif timeStamp=='Season':
        data=GB2
    else :
        data=GB3
        
    return data.plot(kind='bar', stacked=True, figsize=(15,9), 
             title='The number of complaints for each product over Time:' + timeStamp,
             fontsize=17, legend='reverse', rot=-90, sort_columns=True, colormap='gist_ncar')

interact(plot_ProductComplains_overTime,timeStamp=['Year', 'Season', 'Month'] )
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

interactive(children=(Dropdown(description='timeStamp', options=('Year', 'Season', 'Month'), value='Year'), Ou…

# Insights from the plots 

The stacked-bar (histogram) plot can be viewed over years, seasons and months. The goal of this plot was to hypothesize whether the number of complaints over each product is changing over time and if there is a detectable trend based on the year, season or month. 


By looking at the plot over the years:
Apparently during the years, 2 categories namely “checking/saving” is merged with” bank account&service” and the number complaints are almost uniform during the last 6-7 years
Debt collection and credit reporting is growing drastically, in 2017 and 2018 is the majority of complains
Vehicle loan, student loan and prepaid card complains are growing since 2017
Mortgage complaints is shrinking since 2017.


Based on SEASONs in each year, the maximum complains happened in the corresponding season as below which doesn’t reveal any specific pattern:
2012 -> summer ,
2013 -> fall ,
2014 -> spring ,
2015 -> summer ,
2016 ->  fall ,
2017 -> fall ,
2018 ->  spring .


